In [1]:
#!/usr/bin/env python3

In [3]:
%matplotlib inline
import numpy as np 
import time 
from datetime import timedelta
import tensorflow as tf
from readImages import *
from build_tensorflow_graph import *
import matplotlib.pyplot as plt

In [ ]:
firDir = '../data/'

In [2]:
#
## ---------- DCGAN Architecture ----------
#

# Convolution Layer 1
numFilters1 = 32
cnnArchitecture1 = CNN_Architecture(numFilters = numFilters1, 
                                    filterSize = (4, 4), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = 3, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture1 = cnnArchitecture1
dcnnArchitecture1.useReLU = False
# Convolution Layer 2
numFilters2 = 64
cnnArchitecture2 = CNN_Architecture(numFilters = numFilters2, 
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters1, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture2 = cnnArchitecture2
# Convolution Layer 3
numFilters3 = 128
cnnArchitecture3 = CNN_Architecture(numFilters = numFilters3,
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters2, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture3 = cnnArchitecture3

# Convolution Layer 4
numFilters4 = 256
cnnArchitecture3 = CNN_Architecture(numFilters = numFilters3,
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters2, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture4 = cnnArchitecture4

# Fully Connected 1
fc5_size = 1024 

# Lattern Code 
z_dim = 256

# flattern size
flat_size = 4096

NameError: name 'CNN_Architecture' is not defined

In [ ]:
class Image:
    size = 64
    channels = 3

<div>
<h1>Network Output Size</h1>
    <p>
    <ul>
        <li>Input Layer: 64 * 64 * 3 = 12288</li>
        <li>Convolutional Layer1: 32 * 32 * 32 = 32768 </li>
        <li>Convolutional Layer2: 16 * 16 * 64 = 16384 </li>
        <li>Convolutional Layer3: 8 * 8 * 128 = 8192 </li>
        <li>Convolutional Layer4: 4 * 4 * 256 = 4096 </li>
        <li>Fully Connected Layer5: 1024 </li>
        <li>Latten Code: 256 </li>
    </ul>
    </p>
</div>


In [1]:
class DCGAN:
    """
    """
    def __init__(self, batch_size, lr=1e-3):
        self.lr = lr
        self.batch_size = batch_size
        self.sess = tf.Session()
        
        self.build_model()
        
    def build_model(self):
        # _0. placeholder
        self.lattenCode = tf.placeholder(tf.float32, shape = [None, z_dim])
        self.inputImages = tf.placeholder(tf.float32, shape = [None, Image.size, Image.size, Image.channels])
        
        # _1. generator
        self.generatedImgs = self.build_generator(self.lattenCode, is_training = True, bs = self.batch_size, 
                                                  reuse = False, init_stdev = 0.01)
        
        # _2. Discriminator
        self.discriminate_true = self.build_discriminator(self.inputImages, is_training = True, 
                                                          reuse = False, init_stdev = 0.01)
        self.dicriminate_fake = self.build_discriminator(self.generateImgs, is_training = True, 
                                                         reuse = True, init_stdev = 0.01)
        # _3. Cost Function & Optimizer
        # ===== Discriminator loss =====
        discriminator_true_loss = tf.log()
        discriminator_reward = tf.log()
        discriminator_loss = -1.0 * discriminator_reward
        
        # ===== Generator Loss =====
        
        
    def build_test_model(self):
        raise NotImplementedError
        # _4. testing model
            
    def build_discriminator(self, inputLayer, is_training = True, reuse = False, init_stdev = 0.01):
        """
        Arg:
        -------------------------------------------------------------------------
          * inputLayer [tensor] shape = (batch, height, width, channel).
          
        """
        with tf.variable_scope("discriminator", reuse = reuse):
            # Convolutional Layers
            conv1, _ = new_convLayer(inputLayer, cnnArchitecture1, name = "dis_conv1", stdev = init_stdev)
            conv2, _ = new_convLayer(conv1, cnnArchitecture2, name = "dis_conv2", stdev = init_stdev)
            conv2 = bn(conv2, is_training = is_training, scope = "dis_bn2")
            conv3, _ = new_convLayer(conv2, cnnArchitecture3, name = "dis_conv3", stdev = init_stdev)
            conv3 = bn(conv3, is_training = is_training, scope = "dis_bn3")
            conv4, _ = new_convLayer(conv3, cnnArchitecture4, name = "dis_conv4", stdev = init_stdev)
            conv4 = bn(conv4, is_training = is_training, scope = "dis_bn4")
            
            # Flattern
            flat, numNodes = flattenLayer(conv4)
            assert numNodes == flat_size, "Something wrong while flatterning ..."
            
            # Fully Connected Layers
            fc5 = new_fcLayer(flat, numNodes, fc5_size, useReLU=True, name="dis_fc5", stdev=init_stdev)
            fc5 = bn(fc5, is_training = is_training, scope = "dis_bn5")
            output = new_fcLayer(fc5, fc5_size, 1, useReLU=False, name="dis_out", stdev=init_stdev)
            output = tf.nn.sigmoid(output)
            
            return output
            
    
    def build_generator(self, inputLayer, is_training = True, bs = None, reuse = False, init_stdev = 0.01):
        if bs is None:
            bs = self.batch_size
            
        with tf.variable_scope("generator", reuse = reuse):
            # de fully connected layers
            fc1 = new_fcLayer(inputLayer, z_dim, fc5_size, useReLU=True, name="gen_fc1", stdev=init_stdev)
            fc1 = bn(fc1, is_training = is_training, scope = "gen_bn1")
            fc2 = new_fcLayer(fc1, fc5_size, flat_size, useReLU=True, name="gen_dconv2", stdev=init_stdev)
            fc2 = bn(fc2, is_training = is_training, scope = "gen_bn2")
            
            # Reshaping
            conv2 = tf.reshape(fc2, shape=[-1, 4, 4, numFilters4])
            
            # Convolutional Layers
            conv2 = unpool(conv2, kernel=(2, 2))
            conv3 = new_dconvLayer(conv2, dcnnArchitecture4, outputShape = [bs, 8, 8, 128], 
                                   name = "gen_dconv3", stdev = init_stdev)
            conv3 = bn(conv3, is_training = is_training, scope = "gen_bn3")
            
            conv3 = unpool(conv3, kernel=(2, 2))
            conv4 = new_dconvLayer(conv3, dcnnArchitecture3, outputShape = [bs, 16, 16, 64], 
                                   name = "gen_dconv4", stdev = init_stdev)
            conv4 = bn(conv4, is_training = is_training, scope = "gen_bn4")
            
            conv4 = unpool(conv4, kernel=(2, 2))
            conv5 = new_dconvLayer(conv4, dcnnArchitecture2, outputShape = [bs, 32, 32, 32], 
                                   name = "gen_dconv5", stdev = init_stdev)
            conv5 = bn(conv5, is_training = is_training, scope = "gen_bn5")
            
            conv5 = unpool(conv5, kernel=(2, 2))
            image = new_dconvLayer(conv5, dcnnArchitecture1, outputShape = [bs, Image.size, Image.size, Image.channels], 
                                   name = "gen_image", stdev = init_stdev)
            image = tf.nn.sigmoid(image)
            
            return image
            
            
    def train(self, fileDir):
        raise NotImplementedError
        
    def save(self, path):
        raise NotImplementedError
        
    def load(self, path):
        raise NotImplementedError
        
    def generateFakeImages(self, numImgs):
        raise NotImplementedError
        

In [ ]:
if __name__ == '__main__':
    batch_size = 128
    dc_gan = DCGAN(batch_size = batch_size)